In [2]:
import pandas as pd
import numpy as np
from google.colab import files

uploaded = files.upload()

df1994 = pd.read_csv ('adult.csv')
# Drop ? and NaNs
df1994 = df1994.replace("?", np.nan).dropna()
X_1994 = df1994.drop(["income"], axis = 1 )
y_1994 = df1994["income"]
display(X_1994)
display(y_1994)

df2020 = pd.read_csv ('2020_ASEC_version2.csv')
# Drop ? and NaNs
df2020 = df2020.replace("?", np.nan).dropna()
print(df2020)
X_2020 = df2020.drop(["income-50k-threshold","income-90k-threshold","coded-income"], axis = 1 )
y_2020 = df2020["income-90k-threshold"]
display(X_2020)
display(y_2020)

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States
5,34,Private,198693,10th,6,Never-married,Other-service,Not-in-family,White,Male,0,0,30,United-States
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48837,27,Private,257302,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States
48838,40,Private,154374,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States
48839,58,Private,151910,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States
48840,22,Private,201490,HS-grad,9,Never-married,Adm-clerical,Own-child,White,Male,0,0,20,United-States


0        <=50K
1        <=50K
2         >50K
3         >50K
5        <=50K
         ...  
48837    <=50K
48838     >50K
48839    <=50K
48840    <=50K
48841     >50K
Name: income, Length: 45222, dtype: object

       age  workclass  ...  income-50k-threshold income-90k-threshold
0       63  State-gov  ...                  >50k                <=90k
1       64  Local-gov  ...                 <=50k                <=90k
2       54    Private  ...                 <=50k                <=90k
3       37    Private  ...                 <=50k                <=90k
4       35    Private  ...                 <=50k                <=90k
...    ...        ...  ...                   ...                  ...
38571   41    Private  ...                  >50k                 >90k
38572   58    Private  ...                  >50k                 >90k
38573   31    Private  ...                 <=50k                <=90k
38574   57    Private  ...                  >50k                 >90k
38575   32    Private  ...                 <=50k                <=90k

[36254 rows x 16 columns]


,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,hours-per-week,native-country
0,63,State-gov,166860,HS-grad,9,Married-civ-spouse,Exec-managerial,Wife,White,Female,0,40,United-States
1,64,Local-gov,111745,HS-grad,9,Married-civ-spouse,Transport-moving,Husband,White,Male,0,40,United-States
2,54,Private,151825,HS-grad,9,Divorced,Adm-clerical,Not-in-family,White,Female,0,44,United-States
3,37,Private,174690,HS-grad,9,Never-married,Other-service,Own-child,White,Male,0,21,United-States
4,35,Private,197229,Assoc-voc,11,Married-civ-spouse,Prof-specialty,Wife,White,Female,0,40,United-States
...,...,...,...,...,...,...,...,...,...,...,...,...,...
38571,41,Private,180857,HS-grad,9,Married-civ-spouse,Exec-managerial,Husband,Asian-Pac-Islander,Male,0,40,United-States
38572,58,Private,117245,HS-grad,9,Married-civ-spouse,Transport-moving,Husband,Other,Male,0,55,United-States
38573,31,Private,126498,HS-grad,9,Never-married,Other-service,Other-relative,Other,Female,0,40,United-States
38574,57,Private,72015,Bachelors,13,Divorced,Transport-moving,Unmarried,Other,Female,0,50,United-States


0        <=90k
1        <=90k
2        <=90k
3        <=90k
4        <=90k
         ...  
38571     >90k
38572     >90k
38573    <=90k
38574     >90k
38575    <=90k
Name: income-90k-threshold, Length: 36254, dtype: object

In [ ]:
!pip install PyDrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [3]:
# OneHotEncoding
from sklearn.preprocessing import LabelEncoder
X_1994 = X_1994.select_dtypes(exclude=['number']) \
                .apply(LabelEncoder().fit_transform) \
                .join(X_1994.select_dtypes(include=['number']))
display(X_1994)

X_2020 = X_2020.select_dtypes(exclude=['number']) \
                .apply(LabelEncoder().fit_transform) \
                .join(X_2020.select_dtypes(include=['number']))
display(X_2020)

,workclass,education,marital-status,occupation,relationship,race,gender,native-country,age,fnlwgt,educational-num,capital-gain,capital-loss,hours-per-week
0,2,1,4,6,3,2,1,38,25,226802,7,0,0,40
1,2,11,2,4,0,4,1,38,38,89814,9,0,0,50
2,1,7,2,10,0,4,1,38,28,336951,12,0,0,40
3,2,15,2,6,0,2,1,38,44,160323,10,7688,0,40
5,2,0,4,7,1,4,1,38,34,198693,6,0,0,30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48837,2,7,2,12,5,4,0,38,27,257302,12,0,0,38
48838,2,11,2,6,0,4,1,38,40,154374,9,0,0,40
48839,2,11,6,0,4,4,0,38,58,151910,9,0,0,40
48840,2,11,4,0,3,4,1,38,22,201490,9,0,0,20


,workclass,education,marital-status,occupation,relationship,race,gender,native-country,age,fnlwgt,educational-num,capital-gain,hours-per-week
0,5,11,2,4,5,4,0,146,63,166860,9,0,40
1,1,11,2,10,0,4,1,146,64,111745,9,0,40
2,2,11,0,0,1,4,0,146,54,151825,9,0,44
3,2,11,4,6,3,4,1,146,37,174690,9,0,21
4,2,8,2,8,5,4,0,146,35,197229,11,0,40
...,...,...,...,...,...,...,...,...,...,...,...,...,...
38571,2,11,2,4,0,1,1,146,41,180857,9,0,40
38572,2,11,2,10,0,3,1,146,58,117245,9,0,55
38573,2,11,4,6,2,3,0,146,31,126498,9,0,40
38574,2,9,0,10,4,3,0,146,57,72015,13,0,50


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

#Logisitic Regression
logReg = LogisticRegression(max_iter=10000000000)
model = logReg.fit(X_train, y_train)
y_pred = model.predict(X_test)
preds = model.predict_proba(X_test)
print("Logistic Regression")
print("Accuracy:",accuracy_score(y_test, y_pred))
print("ROC: ", roc_auc_score(y_test, preds[:,1]))

#LogReg Tuning
from sklearn.model_selection import GridSearchCV
solvers = ['newton-cg', 'lbfgs', 'liblinear']
penalty = ['l2']
c_values = [100, 10, 1.0, 0.1, 0.01]
grid = dict(solver=solvers,penalty=penalty,C=c_values)
logreg_cv=GridSearchCV(logReg,grid)
logreg_cv.fit(X_train,y_train)

print("LogReg Tuning")
print("Best parameters: ",logreg_cv.best_params_)
print("Accuracy :",logreg_cv.best_score_)
best = logreg_cv.best_estimator
preds = best.predict_proba(X_test)
print("ROC: ", roc_auc_score(y_test, preds[:,1]))

NameError: ignored

In [ ]:
from sklearn.ensemble import RandomForestClassifier

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
# RandomForest
print("Random Forest")
param_grid = {
    'criterion': ["gini", "entropy"],
    'max_features': ["auto", "sqrt", "log2"],
    'n_estimators': [200, 400, 600, 800, 1000]
}

rf = RandomForestClassifier()
grid_search = GridSearchCV(estimator = rf, param_grid = param_grid)
grid_search.fit(X_train, y_train)
preds = grid_search.best_estimator_.predict_proba(X_test)
print(grid_search.best_params_)
rf.fit(X_train, y_train)
print("Accuracy: " + str(rf.score(X_test, y_test)))
print("AUC: " + str(roc_auc_score(y_test, preds[:,1])))

Random Forest
{'criterion': 'entropy', 'max_features': 'sqrt', 'n_estimators': 600}
Accuracy: 0.8575533208089292
AUC: 0.9108165976116789


In [ ]:
#MLP Classifier
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
print("MLP Classifier")
param_grid = {
    'hidden_layer_sizes': [(10,30,10),(20,)],
    'activation': ['tanh', 'relu'],
    'solver': ['sgd', 'adam'],
    'alpha': [0.0001, 0.05],
    'learning_rate': ['constant','adaptive'],
}
mlp = MLPClassifier(max_iter = 1000)
grid_search = GridSearchCV(estimator = mlp, param_grid = param_grid)
grid_search.fit(X_train, y_train)
preds = grid_search.best_estimator_.predict_proba(X_test)
print(grid_search.best_params_)
rf.fit(X_train, y_train)
print("Accuracy: " + str(rf.score(X_test, y_test)))
print("AUC: " + str(roc_auc_score(y_test, preds[:,1])))

MLP Classifier


/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:573: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:573: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Opti

KeyboardInterrupt: ignored

In [4]:
!pip install catboost

     |████████████████████████████████| 67.3MB 52kB/s 


In [5]:
X_1994.head()

,workclass,education,marital-status,occupation,relationship,race,gender,native-country,age,fnlwgt,educational-num,capital-gain,capital-loss,hours-per-week
0,2,1,4,6,3,2,1,38,25,226802,7,0,0,40
1,2,11,2,4,0,4,1,38,38,89814,9,0,0,50
2,1,7,2,10,0,4,1,38,28,336951,12,0,0,40
3,2,15,2,6,0,2,1,38,44,160323,10,7688,0,40
5,2,0,4,7,1,4,1,38,34,198693,6,0,0,30


In [ ]:
from sklearn.model_selection import train_test_split

X_train2020, X_test2020, y_train2020, y_test2020 = train_test_split(X_2020, y_2020, test_size=0.3)
print(X_train2020.shape, X_test2020.shape, y_train2020.shape, y_test2020.shape)

# catboost boosted trees classifier
from catboost import CatBoostClassifier, Pool

# list of the categorical features in X
cat_ftrs = ['workclass','education','marital-status','occupation','relationship','race','gender','native-country']

params = {#'loss_function': ['Logloss', 'CrossEntropy','Poisson'], # not supported with grid_search
          'n_estimators': [500, 1000, 2000],
          'early_stopping_rounds': [300],
          'l2_leaf_reg': [0, 0.1, 1.0, 3.0]
          }

clf1 = CatBoostClassifier(loss_function = 'Logloss', eval_metric='AUC')
train_pool = Pool(X_train2020, y_train2020, cat_features=cat_ftrs)

gs_result = clf1.grid_search(params, train_pool, cv=5)


X_train1994, X_test1994, y_train1994, y_test1994 = train_test_split(X_1994, y_1994, test_size=0.3)
print(X_train1994.shape, X_test1994.shape, y_train1994.shape, y_test1994.shape)

cat_ftrs2 = ['workclass','education','marital-status','occupation','relationship','race','gender','native-country']

params = {#'loss_function': ['Logloss', 'CrossEntropy','Poisson'], # not supported with grid_search
          'n_estimators': [500, 1000, 2000],
          'early_stopping_rounds': [300],
          'l2_leaf_reg': [0, 0.1, 1.0, 3.0]
          }

clf2 = CatBoostClassifier(loss_function = 'Logloss', eval_metric='AUC')
train_pool = Pool(X_train1994, y_train1994, cat_features=cat_ftrs2)

gs_result2 = clf2.grid_search(params, train_pool, cv=5)

Streaming output truncated to the last 5000 lines.
87:	test: 0.9141252	best: 0.9141252 (87)	total: 5.02s	remaining: 1m 49s
88:	test: 0.9142584	best: 0.9142584 (88)	total: 5.07s	remaining: 1m 48s
89:	test: 0.9143031	best: 0.9143031 (89)	total: 5.13s	remaining: 1m 48s
90:	test: 0.9144232	best: 0.9144232 (90)	total: 5.2s	remaining: 1m 49s
91:	test: 0.9145496	best: 0.9145496 (91)	total: 5.25s	remaining: 1m 48s
92:	test: 0.9146163	best: 0.9146163 (92)	total: 5.31s	remaining: 1m 48s
93:	test: 0.9150472	best: 0.9150472 (93)	total: 5.37s	remaining: 1m 48s
94:	test: 0.9150867	best: 0.9150867 (94)	total: 5.44s	remaining: 1m 49s
95:	test: 0.9151106	best: 0.9151106 (95)	total: 5.48s	remaining: 1m 48s
96:	test: 0.9152770	best: 0.9152770 (96)	total: 5.55s	remaining: 1m 48s
97:	test: 0.9153824	best: 0.9153824 (97)	total: 5.61s	remaining: 1m 48s
98:	test: 0.9154999	best: 0.9154999 (98)	total: 5.66s	remaining: 1m 48s
99:	test: 0.9156178	best: 0.9156178 (99)	total: 5.72s	remaining: 1m 48s
100:	test: 0.9

In [6]:
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier, Pool

scores = []
avg_ftrimp = []
# run the model 20 times with 20 different splits
for i in range(20):
  X_train2020, X_test2020, y_train2020, y_test2020 = train_test_split(X_2020, y_2020, test_size=0.3, random_state=i)
  cat_ftrs = ['workclass','education','marital-status','occupation','relationship','race','gender','native-country']
  clf1 = CatBoostClassifier(loss_function = 'Logloss', eval_metric='AUC', n_estimators=1000, early_stopping_rounds=300, l2_leaf_reg=0.1) # best params by gridsearch
  train_pool = Pool(X_train2020, y_train2020, cat_features=cat_ftrs)
  clf1.fit(train_pool)
  scores.append(roc_auc_score(y_test2020, clf1.predict_proba(X_test2020)[:,1]))
  avg_ftrimp.append(clf1.feature_importances_)

print('mean roc score: ', np.mean(scores), ' std dev of roc score: ', np.std(scores))
#print(avg_ftrimp)

"""
X_train2020, X_test2020, y_train2020, y_test2020 = train_test_split(X_2020, y_2020, test_size=0.3, random_state=1)
from catboost import CatBoostClassifier, Pool
# list of the categorical features in X
cat_ftrs = ['workclass','education','marital-status','occupation','relationship','race','gender','native-country']

clf1 = CatBoostClassifier(loss_function = 'Logloss', eval_metric='AUC', n_estimators=1000, early_stopping_rounds=300, l2_leaf_reg=0.1)
train_pool = Pool(X_train2020, y_train2020, cat_features=cat_ftrs)
clf1.fit(train_pool)

# print('2020 best params:', gs_result.get('params')) # best params
print('2020 AUC: ', roc_auc_score(y_test2020, clf1.predict_proba(X_test2020)[:,1]))
feat_importances = pd.Series(clf1.feature_importances_, index=X_2020.columns)
feat_importances.nlargest(20).plot(kind='barh')
plt.title('CatBoost 2020 Feature Importance')
"""


Streaming output truncated to the last 5000 lines.
42:	total: 2.03s	remaining: 45.2s
43:	total: 2.08s	remaining: 45.3s
44:	total: 2.13s	remaining: 45.3s
45:	total: 2.19s	remaining: 45.4s
46:	total: 2.24s	remaining: 45.4s
47:	total: 2.29s	remaining: 45.4s
48:	total: 2.34s	remaining: 45.3s
49:	total: 2.38s	remaining: 45.3s
50:	total: 2.44s	remaining: 45.5s
51:	total: 2.49s	remaining: 45.4s
52:	total: 2.54s	remaining: 45.5s
53:	total: 2.59s	remaining: 45.4s
54:	total: 2.64s	remaining: 45.4s
55:	total: 2.7s	remaining: 45.5s
56:	total: 2.75s	remaining: 45.4s
57:	total: 2.79s	remaining: 45.2s
58:	total: 2.83s	remaining: 45.2s
59:	total: 2.88s	remaining: 45.2s
60:	total: 2.95s	remaining: 45.4s
61:	total: 3s	remaining: 45.4s
62:	total: 3.05s	remaining: 45.4s
63:	total: 3.1s	remaining: 45.4s
64:	total: 3.15s	remaining: 45.3s
65:	total: 3.2s	remaining: 45.3s
66:	total: 3.25s	remaining: 45.3s
67:	total: 3.29s	remaining: 45.2s
68:	total: 3.34s	remaining: 45.1s
69:	total: 3.4s	remaining: 45.2s
70:	

"\nX_train2020, X_test2020, y_train2020, y_test2020 = train_test_split(X_2020, y_2020, test_size=0.3, random_state=1)\nfrom catboost import CatBoostClassifier, Pool\n# list of the categorical features in X\ncat_ftrs = ['workclass','education','marital-status','occupation','relationship','race','gender','native-country']\n\nclf1 = CatBoostClassifier(loss_function = 'Logloss', eval_metric='AUC', n_estimators=1000, early_stopping_rounds=300, l2_leaf_reg=0.1)\ntrain_pool = Pool(X_train2020, y_train2020, cat_features=cat_ftrs)\nclf1.fit(train_pool)\n\n# print('2020 best params:', gs_result.get('params')) # best params\nprint('2020 AUC: ', roc_auc_score(y_test2020, clf1.predict_proba(X_test2020)[:,1]))\nfeat_importances = pd.Series(clf1.feature_importances_, index=X_2020.columns)\nfeat_importances.nlargest(20).plot(kind='barh')\nplt.title('CatBoost 2020 Feature Importance')\n"

In [8]:
print('mean roc score for 2020: ', np.mean(scores), ' std dev of roc score: ', np.std(scores))
print('mean ftr importance for 2020: ', np.mean(avg_ftrimp,axis=0))

mean roc score for 2020:  0.8656754948900913  std dev of roc score:  0.003116038975595963
mean ftr importance for 2020:  [ 6.64472214  9.38434813  4.01519676 13.32331756  9.24531242  3.7675681
  4.11704864  4.47995207 14.01629577  7.21148678  5.58997651  3.62475389
 14.58002124]


In [10]:
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier, Pool

scores94 = []
avg_ftrimp94 = []
# run the model 20 times with 20 different splits
for i in range(20):
  X_train1994, X_test1994, y_train1994, y_test1994 = train_test_split(X_1994, y_1994, test_size=0.3, random_state=i) # random state 1 through 20
  cat_ftrs = ['workclass','education','marital-status','occupation','relationship','race','gender','native-country']
  clf1 = CatBoostClassifier(loss_function = 'Logloss', eval_metric='AUC', n_estimators=1000, early_stopping_rounds=300, l2_leaf_reg=0.1) # best params by gridsearch
  train_pool = Pool(X_train1994, y_train1994, cat_features=cat_ftrs)
  clf1.fit(train_pool)
  scores94.append(roc_auc_score(y_test1994, clf1.predict_proba(X_test1994)[:,1]))
  avg_ftrimp94.append(clf1.feature_importances_)

"""
X_train1994, X_test1994, y_train1994, y_test1994 = train_test_split(X_1994, y_1994, test_size=0.3, random_state=1)
from catboost import CatBoostClassifier, Pool
# list of the categorical features in X
cat_ftrs = ['workclass','education','marital-status','occupation','relationship','race','gender','native-country']

clf2 = CatBoostClassifier(loss_function = 'Logloss', eval_metric='AUC', n_estimators=1000, early_stopping_rounds=300, l2_leaf_reg=0.1)
train_pool = Pool(X_train1994, y_train1994, cat_features=cat_ftrs)
clf2.fit(train_pool)

print('1994 AUC: ', roc_auc_score(y_test1994, clf2.predict_proba(X_test1994)[:,1]))
#print('1994 best params:', gs_result2.get('params')) # best params
feat_importances = pd.Series(clf2.feature_importances_, index=X_1994.columns)
feat_importances.nlargest(20).plot(kind='barh')
plt.title('CatBoost 1994 Feature Importance')
"""

Streaming output truncated to the last 5000 lines.
0:	total: 67.3ms	remaining: 1m 7s
1:	total: 128ms	remaining: 1m 4s
2:	total: 193ms	remaining: 1m 4s
3:	total: 254ms	remaining: 1m 3s
4:	total: 322ms	remaining: 1m 4s
5:	total: 385ms	remaining: 1m 3s
6:	total: 448ms	remaining: 1m 3s
7:	total: 507ms	remaining: 1m 2s
8:	total: 572ms	remaining: 1m 3s
9:	total: 632ms	remaining: 1m 2s
10:	total: 688ms	remaining: 1m 1s
11:	total: 739ms	remaining: 1m
12:	total: 802ms	remaining: 1m
13:	total: 863ms	remaining: 1m
14:	total: 918ms	remaining: 1m
15:	total: 981ms	remaining: 1m
16:	total: 1.03s	remaining: 59.4s
17:	total: 1.08s	remaining: 59.1s
18:	total: 1.13s	remaining: 58.4s
19:	total: 1.18s	remaining: 57.8s
20:	total: 1.25s	remaining: 58.2s
21:	total: 1.29s	remaining: 57.5s
22:	total: 1.35s	remaining: 57.3s
23:	total: 1.4s	remaining: 56.8s
24:	total: 1.46s	remaining: 57s
25:	total: 1.5s	remaining: 56.4s
26:	total: 1.55s	remaining: 56s
27:	total: 1.6s	remaining: 55.4s
28:	total: 1.65s	remaining: 

"\nX_train1994, X_test1994, y_train1994, y_test1994 = train_test_split(X_1994, y_1994, test_size=0.3, random_state=1)\nfrom catboost import CatBoostClassifier, Pool\n# list of the categorical features in X\ncat_ftrs = ['workclass','education','marital-status','occupation','relationship','race','gender','native-country']\n\nclf2 = CatBoostClassifier(loss_function = 'Logloss', eval_metric='AUC', n_estimators=1000, early_stopping_rounds=300, l2_leaf_reg=0.1)\ntrain_pool = Pool(X_train1994, y_train1994, cat_features=cat_ftrs)\nclf2.fit(train_pool)\n\nprint('1994 AUC: ', roc_auc_score(y_test1994, clf2.predict_proba(X_test1994)[:,1]))\n#print('1994 best params:', gs_result2.get('params')) # best params\nfeat_importances = pd.Series(clf2.feature_importances_, index=X_1994.columns)\nfeat_importances.nlargest(20).plot(kind='barh')\nplt.title('CatBoost 1994 Feature Importance')\n"

In [11]:
print('mean roc score for 1994: ', np.mean(scores94), ' std dev of roc score: ', np.std(scores94))
print('mean ftr importance for 1994: ', np.mean(avg_ftrimp94,axis=0))

mean roc score for 1994:  0.9283559938846778  std dev of roc score:  0.0017174168032758296
mean ftr importance for 1994:  [ 3.07600078  4.69759536  6.20142249  7.19116268 13.90605886  1.44848259
  1.58444659  1.92935704 10.92672278  3.93177687  4.53510564 23.19285397
  9.25049731  8.12851703]


In [ ]:
#XGBoost 
from sklearn.model_selection import train_test_split, GridSearchCV

X_train2020, X_test2020, y_train2020, y_test2020 = train_test_split(X_2020, y_2020, test_size=0.3)
print(X_train2020.shape, X_test2020.shape, y_train2020.shape, y_test2020.shape)

# xgboost boosted trees classifier
from xgboost import XGBClassifier

params = {'objective': ['binary:logistic'],
          'n_estimators': [500, 1000, 2000],
          'max_depth': [4,6,8],
          'min_split_loss': [0, 1],
          'subsample': [0.7, 1]
          }

clf3 = XGBClassifier(eval_metric='auc', eta=0.1)

gs_result = GridSearchCV(clf3, params, scoring='roc_auc', cv=5).fit(X_train2020, y_train2020)
clf3 = gs_result.best_estimator_


X_train1994, X_test1994, y_train1994, y_test1994 = train_test_split(X_1994, y_1994, test_size=0.3)
print(X_train1994.shape, X_test1994.shape, y_train1994.shape, y_test1994.shape)

params = {'objective': ['binary:logistic'],
          'n_estimators': [500, 1000, 2000],
          'max_depth': [4,6,8],
          'min_split_loss': [0, 1],
          'subsample': [0.7, 1]
          }

clf4 = XGBClassifier(eval_metric='auc', eta=0.1)

gs_result2 = GridSearchCV(clf4, params, scoring='roc_auc', cv=5).fit(X_train1994, y_train1994)
clf4 = gs_result2.best_estimator_

(25377, 13) (10877, 13) (25377,) (10877,)
(31655, 14) (13567, 14) (31655,) (13567,)


In [13]:
from sklearn.model_selection import train_test_split, GridSearchCV
from xgboost import XGBClassifier
#avg of 20 runs
xgbscores20 = []
xgbavg_ftrimp20 = []
for i in range(20):
  X_train2020, X_test2020, y_train2020, y_test2020 = train_test_split(X_2020, y_2020, test_size=0.3, random_state=i)
  clf3 = XGBClassifier(objective='binary:logistic', eval_metric='auc', eta=0.1, n_estimators=500, max_depth=4, min_split_loss=1, subsample=1).fit(X_train2020, y_train2020)
  xgbscores20.append(roc_auc_score(y_test2020, clf3.predict_proba(X_test2020)[:,1]))
  xgbavg_ftrimp20.append(clf3.feature_importances_)

"""
import matplotlib.pyplot as plt
#print(gs_result.best_params_)
print('2020: ', roc_auc_score(y_test2020, clf3.predict_proba(X_test2020)[:,1]))
feat_importances = pd.Series(clf3.feature_importances_, index=X_2020.columns)
feat_importances.nlargest(20).plot(kind='barh')
plt.title('XGBoost 2020 Feature Importance')
print(clf3.feature_importances_)
"""

"\nimport matplotlib.pyplot as plt\n#print(gs_result.best_params_)\nprint('2020: ', roc_auc_score(y_test2020, clf3.predict_proba(X_test2020)[:,1]))\nfeat_importances = pd.Series(clf3.feature_importances_, index=X_2020.columns)\nfeat_importances.nlargest(20).plot(kind='barh')\nplt.title('XGBoost 2020 Feature Importance')\nprint(clf3.feature_importances_)\n"

In [17]:
print('mean roc score for 2020: ', np.mean(xgbscores20), ' std dev of roc score: ', np.std(xgbscores20))
print('mean ftr importance for 2020: ', np.mean(xgbavg_ftrimp20,axis=0))

mean roc score for 2020:  0.8625016229204119  std dev of roc score:  0.0034781062833005288
mean ftr importance for 2020:  [0.04106471 0.0332661  0.02809573 0.06678764 0.20714095 0.01963918
 0.07698719 0.02014585 0.06059478 0.01754834 0.28569523 0.07031722
 0.07271709]


In [15]:
from sklearn.model_selection import train_test_split, GridSearchCV
from xgboost import XGBClassifier

xgbscores94 = []
xgbavg_ftrimp94 = []
for i in range(20):
  X_train1994, X_test1994, y_train1994, y_test1994 = train_test_split(X_1994, y_1994, test_size=0.3, random_state=i)
  clf4 = XGBClassifier(objective='binary:logistic', eval_metric='auc', eta=0.1, n_estimators=500, max_depth=4, min_split_loss=0, subsample=1).fit(X_train1994, y_train1994)
  xgbscores94.append(roc_auc_score(y_test1994, clf4.predict_proba(X_test1994)[:,1]))
  xgbavg_ftrimp94.append(clf4.feature_importances_)
X_train1994, X_test1994, y_train1994, y_test1994 = train_test_split(X_1994, y_1994, test_size=0.3)

clf4 = XGBClassifier(objective='binary:logistic', eval_metric='auc', eta=0.1, n_estimators=500, max_depth=4, min_split_loss=0, subsample=1).fit(X_train1994, y_train1994)



"""
import matplotlib.pyplot as plt
#print(gs_result2.best_params_)
print('1994: ', roc_auc_score(y_test1994, clf4.predict_proba(X_test1994)[:,1]))
feat_importances = pd.Series(clf4.feature_importances_, index=X_1994.columns)
feat_importances.nlargest(20).plot(kind='barh')
plt.title('XGBoost 1994 Feature Importance')
"""

"\nimport matplotlib.pyplot as plt\n#print(gs_result2.best_params_)\nprint('1994: ', roc_auc_score(y_test1994, clf4.predict_proba(X_test1994)[:,1]))\nfeat_importances = pd.Series(clf4.feature_importances_, index=X_1994.columns)\nfeat_importances.nlargest(20).plot(kind='barh')\nplt.title('XGBoost 1994 Feature Importance')\n"

In [16]:
print('mean roc score for 1994: ', np.mean(xgbscores94), ' std dev of roc score: ', np.std(xgbscores94))
print('mean ftr importance for 1994: ', np.mean(xgbavg_ftrimp94,axis=0))

mean roc score for 1994:  0.9275766757408904  std dev of roc score:  0.0016874834467352805
mean ftr importance for 1994:  [0.01650579 0.00927002 0.10517808 0.03336225 0.38843971 0.01130026
 0.02919816 0.01028824 0.04184995 0.01035112 0.1580036  0.10415275
 0.05093725 0.03116281]


In [ ]:
# Catboost trained on 1994 and tested on 2020, possibly vice versa
# best params: 'od_wait': 300, 'iterations': 1000, 'l2_leaf_reg': 0.1

X_1994_d = X_1994.drop('capital-loss', axis=1)

from catboost import CatBoostClassifier, Pool

# list of the categorical features in X
cat_ftrs = ['workclass','education','marital-status','occupation','relationship','race','gender','native-country']

clf1 = CatBoostClassifier(loss_function = 'Logloss', eval_metric='AUC', n_estimators=1000, early_stopping_rounds=300, l2_leaf_reg=0.1)
train_pool = Pool(X_1994_d, y_1994, cat_features=cat_ftrs)
clf1.fit(train_pool)

0:	total: 146ms	remaining: 2m 26s
1:	total: 234ms	remaining: 1m 56s
2:	total: 325ms	remaining: 1m 48s
3:	total: 411ms	remaining: 1m 42s
4:	total: 499ms	remaining: 1m 39s
5:	total: 574ms	remaining: 1m 35s
6:	total: 657ms	remaining: 1m 33s
7:	total: 743ms	remaining: 1m 32s
8:	total: 808ms	remaining: 1m 28s
9:	total: 878ms	remaining: 1m 26s
10:	total: 941ms	remaining: 1m 24s
11:	total: 1s	remaining: 1m 22s
12:	total: 1.07s	remaining: 1m 21s
13:	total: 1.16s	remaining: 1m 21s
14:	total: 1.24s	remaining: 1m 21s
15:	total: 1.31s	remaining: 1m 20s
16:	total: 1.39s	remaining: 1m 20s
17:	total: 1.47s	remaining: 1m 20s
18:	total: 1.53s	remaining: 1m 19s
19:	total: 1.61s	remaining: 1m 18s
20:	total: 1.68s	remaining: 1m 18s
21:	total: 1.75s	remaining: 1m 17s
22:	total: 1.84s	remaining: 1m 18s
23:	total: 1.93s	remaining: 1m 18s
24:	total: 2.01s	remaining: 1m 18s
25:	total: 2.08s	remaining: 1m 18s
26:	total: 2.16s	remaining: 1m 17s
27:	total: 2.23s	remaining: 1m 17s
28:	total: 2.31s	remaining: 1m 17

array([0.06775962, 0.1993935 , 0.04929673, ..., 0.00222389, 0.14111702,
       0.0656487 ])

In [ ]:
from sklearn.metrics import roc_auc_score
print(roc_auc_score(y_2020, clf1.predict_proba(X_2020)[:,1]))

0.7323438016378179


In [ ]:
X_1994_d = X_1994.drop('capital-loss', axis=1)

from catboost import CatBoostClassifier, Pool

# list of the categorical features in X
cat_ftrs = ['workclass','education','marital-status','occupation','relationship','race','gender','native-country']

clf2 = CatBoostClassifier(loss_function = 'Logloss', eval_metric='AUC', n_estimators=1000, early_stopping_rounds=300, l2_leaf_reg=0.1)
train_pool = Pool(X_2020, y_2020, cat_features=cat_ftrs)
clf2.fit(train_pool)

0:	total: 78.3ms	remaining: 1m 18s
1:	total: 148ms	remaining: 1m 14s
2:	total: 205ms	remaining: 1m 8s
3:	total: 275ms	remaining: 1m 8s
4:	total: 335ms	remaining: 1m 6s
5:	total: 401ms	remaining: 1m 6s
6:	total: 475ms	remaining: 1m 7s
7:	total: 547ms	remaining: 1m 7s
8:	total: 604ms	remaining: 1m 6s
9:	total: 678ms	remaining: 1m 7s
10:	total: 735ms	remaining: 1m 6s
11:	total: 792ms	remaining: 1m 5s
12:	total: 844ms	remaining: 1m 4s
13:	total: 903ms	remaining: 1m 3s
14:	total: 974ms	remaining: 1m 3s
15:	total: 1.03s	remaining: 1m 3s
16:	total: 1.1s	remaining: 1m 3s
17:	total: 1.17s	remaining: 1m 4s
18:	total: 1.24s	remaining: 1m 4s
19:	total: 1.3s	remaining: 1m 3s
20:	total: 1.36s	remaining: 1m 3s
21:	total: 1.41s	remaining: 1m 2s
22:	total: 1.47s	remaining: 1m 2s
23:	total: 1.52s	remaining: 1m 1s
24:	total: 1.59s	remaining: 1m 1s
25:	total: 1.65s	remaining: 1m 1s
26:	total: 1.71s	remaining: 1m 1s
27:	total: 1.78s	remaining: 1m 1s
28:	total: 1.85s	remaining: 1m 1s
29:	total: 1.91s	remain

In [ ]:
from sklearn.metrics import roc_auc_score
print(roc_auc_score(y_1994, clf2.predict_proba(X_1994)[:,1]))

0.8390541507512945


In [ ]:
# Catboost trained on 1994 and tested on 2020, using 2020 50k label
# best params: 'od_wait': 300, 'iterations': 1000, 'l2_leaf_reg': 0.1

X_1994_d = X_1994.drop('capital-loss', axis=1)
y_2020_50 = df2020["income-50k-threshold"]

from catboost import CatBoostClassifier, Pool

# list of the categorical features in X
cat_ftrs = ['workclass','education','marital-status','occupation','relationship','race','gender','native-country']

clf3 = CatBoostClassifier(loss_function = 'Logloss', eval_metric='AUC', n_estimators=1000, early_stopping_rounds=300, l2_leaf_reg=0.1)
train_pool = Pool(X_1994_d, y_1994, cat_features=cat_ftrs)
clf3.fit(train_pool)

0:	total: 128ms	remaining: 2m 7s
1:	total: 220ms	remaining: 1m 50s
2:	total: 307ms	remaining: 1m 42s
3:	total: 386ms	remaining: 1m 36s
4:	total: 470ms	remaining: 1m 33s
5:	total: 546ms	remaining: 1m 30s
6:	total: 626ms	remaining: 1m 28s
7:	total: 704ms	remaining: 1m 27s
8:	total: 765ms	remaining: 1m 24s
9:	total: 845ms	remaining: 1m 23s
10:	total: 919ms	remaining: 1m 22s
11:	total: 983ms	remaining: 1m 20s
12:	total: 1.06s	remaining: 1m 20s
13:	total: 1.14s	remaining: 1m 19s
14:	total: 1.21s	remaining: 1m 19s
15:	total: 1.28s	remaining: 1m 18s
16:	total: 1.34s	remaining: 1m 17s
17:	total: 1.42s	remaining: 1m 17s
18:	total: 1.48s	remaining: 1m 16s
19:	total: 1.56s	remaining: 1m 16s
20:	total: 1.63s	remaining: 1m 15s
21:	total: 1.7s	remaining: 1m 15s
22:	total: 1.78s	remaining: 1m 15s
23:	total: 1.87s	remaining: 1m 15s
24:	total: 1.94s	remaining: 1m 15s
25:	total: 2.01s	remaining: 1m 15s
26:	total: 2.09s	remaining: 1m 15s
27:	total: 2.16s	remaining: 1m 14s
28:	total: 2.23s	remaining: 1m 1

In [ ]:
from sklearn.metrics import roc_auc_score
print(roc_auc_score(y_2020_50, clf3.predict_proba(X_2020)[:,1]))

0.7293243422607706


In [ ]:

# Use scikit-learn to grid search the batch size and epochs
import numpy
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import train_test_split, GridSearchCV

X_train1994, X_test1994, y_train1994, y_test1994 = train_test_split(X_1994, y_1994, test_size=0.3)
print(X_train1994.shape)
print(y_train1994.shape)
# Function to create model, required for KerasClassifier
def create_model():
	# create model
	model = Sequential()
	model.add(Dense(12, input_dim=14, activation='relu'))
	model.add(Dense(1, activation='sigmoid'))
	# Compile model
	model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
	return model
# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)

# create model
model = KerasClassifier(build_fn=create_model, verbose=0)
# define the grid search parameters
batch_size = [10, 20, 40, 60, 80, 100]
epochs = [10, 50, 100]
param_grid = dict(batch_size=batch_size, epochs=epochs)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X_train1994, y_train1994)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))


(31655, 14)
(31655,)
Best: 0.787459 using {'batch_size': 20, 'epochs': 100}
0.619735 (0.215317) with: {'batch_size': 10, 'epochs': 10}
0.779277 (0.022565) with: {'batch_size': 10, 'epochs': 50}
0.626887 (0.201119) with: {'batch_size': 10, 'epochs': 100}
0.771694 (0.022628) with: {'batch_size': 20, 'epochs': 10}
0.786574 (0.005198) with: {'batch_size': 20, 'epochs': 50}
0.787459 (0.012316) with: {'batch_size': 20, 'epochs': 100}
0.613747 (0.250409) with: {'batch_size': 40, 'epochs': 10}
0.768125 (0.010579) with: {'batch_size': 40, 'epochs': 50}
0.768220 (0.031777) with: {'batch_size': 40, 'epochs': 100}
0.605801 (0.251569) with: {'batch_size': 60, 'epochs': 10}
0.631677 (0.195688) with: {'batch_size': 60, 'epochs': 50}
0.592031 (0.190606) with: {'batch_size': 60, 'epochs': 100}
0.779845 (0.016874) with: {'batch_size': 80, 'epochs': 10}
0.763322 (0.029915) with: {'batch_size': 80, 'epochs': 50}
0.775549 (0.010451) with: {'batch_size': 80, 'epochs': 100}
0.579565 (0.240765) with: {'batch_